In [1]:
import os
from tkinter import *
from tkinter import messagebox as ms
filename="newfile1.tsv"
def action():
    ms.showinfo(title="Success",message="Data updated!")
def spam():
    status="spam"
    message=str(mtc.get())
    f=open(filename,"a")
    f.write(status+"\t"+message+"\n")  
    f.close()
def ham():
    status="ham"
    message=str(mtc.get())
    f=open(filename,"a")
    f.write(status+"\t"+message+"\n") 
    f.close()
root=Tk()
root.title("Receiving your data")
root.geometry("600x500+190+20")
root.configure(background="black")
mtc=StringVar()
heading=Label(root,text="SPAM DETECTION",font=("vardana",20,"bold"), fg="white",bg="black").place(x=180,y=40)
l1=Label(root,text="Enter your mail :",font=("calibri",12,"bold"),fg="red",bg="black").place(x=240,y=120)
t1=Entry(root,textvariable=mtc,width=96,bg="red").place(x=9,y=160)
l2=Label(root,text="Choose target class :",font=("calibri",12,"bold"),fg="red",bg="black").place(x=230,y=230)
r1=Radiobutton(root,text="Spam",bg="black",fg="red",font=("calibri",12,"bold"),value=1,command=spam).place(x=120,y=259)
r2=Radiobutton(root,text="Ham",bg="black",fg="red",font=("calibri",12,"bold"),value=2,command=ham).place(x=410,y=259)
b=Button(root,text="Submit Record",bg="red",command=action).place(x=255,y=330)
root.mainloop() 


In [2]:
import pandas as pd
data=pd.read_csv('data/sms.tsv',header=None,delimiter="\t",names=["TARGET","MESSAGE"])
data1=pd.read_csv(filename,header=None,delimiter="\t",names=["TARGET","MESSAGE"])

In [3]:
import matplotlib.pyplot as plt
data.TARGET.value_counts().plot(kind="bar")
plt.title("Spam data collected")
plt.show()
print(data.TARGET.value_counts())
print("------------------------------------------------------")
data1.TARGET.value_counts().plot(kind="bar")
plt.title("Data fetched from the user")
plt.show()
print(data1.TARGET.value_counts())

<Figure size 640x480 with 1 Axes>

ham     4827
spam     754
Name: TARGET, dtype: int64
------------------------------------------------------


<Figure size 640x480 with 1 Axes>

spam    1
Name: TARGET, dtype: int64


In [4]:
data["TARGET"]=data["TARGET"].map({"spam":1,"ham":0})
data1["TARGET"]=data1["TARGET"].map({"spam":1,"ham":0})

In [5]:
data1

,TARGET,MESSAGE
0,1,congratulations


In [6]:
x=data.MESSAGE
y=data.TARGET
x1=data1.MESSAGE
y1=data1.TARGET

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
c=CountVectorizer(stop_words="english")
c1=CountVectorizer(stop_words="english")
x_new=c.fit_transform(x)
x1_new=c1.fit_transform(x1)

In [8]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [9]:
x_train=x
x_test=x1
y_train=y
y_test=y1

In [10]:
pipe1=make_pipeline(CountVectorizer(),MultinomialNB())
pipe2=make_pipeline(CountVectorizer(),LogisticRegression())

In [11]:
pipe1.fit(x_train,y_train)
pipe2.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [12]:
pred1=pipe1.predict(x_test)
pred2=pipe2.predict(x_test)

In [13]:
fwrite=open("naiveBayes.tsv","w")
for i in range(0,len(x1)):
#first for Naive bias, i.e pipe1
    s=str(x1[i])
    if pred1[i]==0:
        fwrite.write("ham"+"\t"+s+"\n")
    else:
        fwrite.write("spam"+"\t"+s+"\n")
fwrite.close()
fwrite=open("logisticRegression.tsv","w")
for i in range(0,len(x1)):
#first for Logistic Regression, i.e pipe2
    s=str(x1[i])
    if pred2[i]==0:
        fwrite.write("ham"+"\t"+s+"\n")
    else:
        fwrite.write("spam"+"\t"+s+"\n")
fwrite.close()

In [14]:
print("-------------------THE ACTUAL DATA-----------------\n")
dispOriginal=pd.read_csv(filename,header=None,delimiter="\t",names=["TARGET","MESSAGE"])
dispOriginal

-------------------THE ACTUAL DATA-----------------



,TARGET,MESSAGE
0,spam,congratulations


In [15]:
print("-----------------THE PREDICTED DATA---------------\n")
print("1. For Naive Bayes :-")
dispPrediction1=pd.read_csv("naiveBayes.tsv",header=None,delimiter="\t",names=["TARGET","MESSAGE"])
dispPrediction1

-----------------THE PREDICTED DATA---------------

1. For Naive Bayes :-


,TARGET,MESSAGE
0,spam,congratulations


In [16]:
print("-----------------THE PREDICTED DATA---------------\n")
print("2. For Logistic Regression :-")
dispPrediction2=pd.read_csv("logisticRegression.tsv",header=None,delimiter="\t",names=["TARGET","MESSAGE"])
dispPrediction2

-----------------THE PREDICTED DATA---------------

2. For Logistic Regression :-


,TARGET,MESSAGE
0,ham,congratulations


In [17]:
from sklearn.metrics import accuracy_score
r1=(accuracy_score(y_test,pred1))*100
r2=(accuracy_score(y_test,pred2))*100
s="Score for Naive bayes = "+str(r1)+"%"+"\nScore for Logistic Regression = "+str(r2)+"%"
ms.showinfo(title="Predicted Data",message=s)

root.mainloop()

In [18]:
a=[]
y_test=np.array(y_test)
for i in range(0,len(y_test)):
    if(y_test[i]!=pred1[i]):
        a.append(i)

if len(a)==0:
    ms.showinfo(title="Prediction by Naive Bayes",message="No wrong predcition found")
    root.mainloop()
else:
    for i in range(0,len(a)):
        var="Message you entered at '"+str(a[i]+1)+"' position was missclassified\nThe Message was = '"+str(x1[a[i]])+"'"
        ms.showinfo(title="Prediction by Naive Bayes",message=var)
        root.mainloop()

In [19]:
b=[]
for i in range(0,len(y_test)):
    if(y_test[i]!=pred2[i]):
        b.append(i)      
if len(b)==0:
    ms.showinfo(title="Prediction by Logisitic Regression",message="No wrong predcition found")
    root.mainloop()
else:
    for i in range(0,len(b)):
        var="Message you entered at '"+str(b[i]+1)+"' position was missclassified\nThe Message was = '"+str(x1[b[i]])+"'"
        ms.showinfo(title="Prediction by Logisitic Regression",message=var)
        root.mainloop()
        


In [ ]:
print(b)